In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append("python_lib")

In [3]:
from tqdm.notebook import tqdm_notebook as tqdm
import nnsplit
from nnsplit import train, utils
train.tqdm = tqdm

In [4]:
cache_dir = Path("cache")
cache_dir.mkdir(exist_ok=True)

# Prepare data

In [7]:
train.prepare_data("train_data/dewiki-20180920-corpus.xml", "de", max_n_sentences=1_000_000, 
                   data_directory=cache / "de_data")

(tensor([[107, 106, 111,  ...,  86,  99, 105],
         [117,  34,  82,  ..., 102, 119, 116],
         [103, 112,  34,  ...,  71, 107, 112],
         ...,
         [ 99, 118, 107,  ..., 116, 111,  99],
         [106, 112, 107,  ..., 112, 113, 110],
         [105, 103, 117,  ..., 114, 106, 103]], dtype=torch.uint8),
 tensor([[[False, False],
          [False, False],
          [False, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [ True, False],
          [False, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [False, False],
          [ True, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         ...,
 
         [[False, False],
          [False, False],
          [False, False],
          ...,
          [False, False],
          [False, False],


# Train model

In [10]:
learner = train.train_from_directory("cache/de_data", n_epochs=2)
learner

epoch,train_loss,valid_loss,time
0,0.003551,0.003577,01:30
1,0.002744,0.002848,01:28


Learner(data=DataBunch;

Train: <torch.utils.data.dataset.TensorDataset object at 0x7f0465feb450>;

Valid: <torch.utils.data.dataset.TensorDataset object at 0x7f0465fd03d0>;

Test: None, model=Network(
  (embedding): Embedding(129, 25)
  (lstm1): LSTM(25, 50, bias=False, batch_first=True, bidirectional=True)
  (lstm2): LSTM(100, 50, bias=False, batch_first=True, bidirectional=True)
  (out): Linear(in_features=100, out_features=2, bias=True)
), opt_func=functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99)), loss_func=<function loss at 0x7f0465fce560>, metrics=[], true_wd=True, bn_wd=True, wd=0.01, train_bn=True, path=PosixPath('.'), model_dir='models', callback_fns=[functools.partial(<class 'fastai.basic_train.Recorder'>, add_time=True, silent=False)], callbacks=[], layer_groups=[Sequential(
  (0): Embedding(129, 25)
  (1): LSTM(25, 50, bias=False, batch_first=True, bidirectional=True)
  (2): LSTM(100, 50, bias=False, batch_first=True, bidirectional=True)
  (3): Linear(i

In [11]:
utils.store_model(learner, "data/de")

# Test

In [5]:
from nnsplit import NNSplit

In [6]:
splitter = NNSplit(utils.load_model("cache/de_data"))

In [9]:
splitter.split(["Das ist ein Test Das ist noch ein Test."])

[[[Token(text='Das', whitespace=' '),
   Token(text='ist', whitespace=' '),
   Token(text='ein', whitespace=' '),
   Token(text='Test', whitespace=' ')],
  [Token(text='Das', whitespace=' '),
   Token(text='ist', whitespace=' '),
   Token(text='noch', whitespace=' '),
   Token(text='ein', whitespace=' '),
   Token(text='Test', whitespace='')],
  [Token(text='.', whitespace='')]]]